## 改写AI大模型推理新纪元o1模型的API调用与总结

当我们都在翘首期盼GPT-5时，OpenAI却给了我们一个大模型的转折点 —— 全新的o1模型！

OpenAI不走寻常路，抛开GPT系列的命名，将计数器重置回1。这个名为o1的新模型，不仅标志着全新产品线的诞生，更代表着AI领域的一次重大突破。

**为什么o1如此特别？**

想象一下，一个达到"博士级推理能力"的AI，o1在数学、编程和解谜等领域展现出惊人的表现。虽然它可能比GPT-4o要"慢"一些，但这是有原因的 —— 因为它在认真思考！

引入"推理令牌"机制，让o1能够像人类一样在给出答案前进行深入思考。这种方法使模型能够：

- 在生成最终答案前考虑多种可能性
- 分析问题的不同角度，验证推理过程的正确性

**实用性解析**

- 需要快速回应，处理图像？选GPT-4o、GPT-4o mini正合适
- 但如果你需要深度推理，愿意等待更优质的答案，o1就是不二之选


 两个版本供选择

1. **o1-preview**
   - 完整版o1模型预览版（还不是最终的o1，但是目前最智能最贵）
   - 专攻需要广泛常识的复杂问题

2. **o1-mini**
   - 更快速、性价比更高
   - 特别适合编码、数学和科学任务

 ⚠️ 测试版限制要点
 
- 仅支持文本输入输出，图像处理尚不可用。
- 不支持系统消息，仅允许用户和助理消息。
- 暂不支持流式处理
- 工具和函数调用暂未开放
- 其他参数固定，temperature 、 top_p和n等参数固定为1，而presence_penalty和frequency_penalty固定为0。
- 助手和 Batch API ：o1 模型尚未与 Assistants API 或 Batch API 集成。

OpenAI 计划随着 o1 退出测试版而逐步添加对其中一些参数和功能的支持。多模态和工具使用等更高级的功能预计将包含在 o1 系列的未来迭代中。

 推理token：o1的核心创新
- 模型通过"reasoning tokens"进行深度思考
- 虽然用户看不到推理过程，但会影响计费
- 支持128,000个令牌的超大上下文窗口

 使用建议
- 避免过度引导思维链，由于模型内部进行推理，“逐步思考”或“解释推理”的指令是不必要的。
- 善用分隔符构造输入，使用三引号、XML 标签、节标题等分隔符明确区分输入的不同部分，使模型更容易正确解释各个部分。
- 限制检索增强（RAG）中的附加上下文：当提供额外的上下文或文档时，只包含最相关的部分，以防止模型做出过度复杂的响应。

开发者可以通过聊天完成端点轻松接入，只需设置`model="o1-preview"`即可开始体验这一革命性的AI推理能力。

api调用限制——3级及以上用户


![](https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/LingYi/20241025150051.png)

https://help.openai.com/en/articles/9824962-openai-o1-preview-and-o1-mini-usage-limits-on-chatgpt-and-the-api

### o1模型的API调用详解与总结

获取 API 凭证

如果您有一个3级帐户，您可以从 OpenAI 的API 密钥页面获取您的秘密 API 密钥.

导入openai库，初始化客户端对象,请注意，如果您运行的是旧版本，则可能需要更新openai库。

In [ ]:
!pip install openai

In [ ]:
from openai import OpenAI
# 如果成功配置了环境变量的在OPENAI提供的工具包中会自动找到我们的秘钥
client = OpenAI()
# 如果没有配置环境变量也可以用变量显式传入 但注意保护好api key不被别人看见
# client = OpenAI(api_key="bjhgsdjhfkad_your_api_key_here") 

#### API 调用参数

函数和之前的GPT4o系列所用函数（即背后的端点ulr）一样

且返回对象结构一致

但只能使用三个参数，两个必须参数以及max_completion_tokens

 model参数

model参数表示的是当前对话调用的模型，具体模型列表参考OpenAI官网Limits页面中的Chat类模型：https://platform.openai.com/account/limits

目前o1系列模型只有**o1-preview**和**o1-mini**下面两个版本


![](https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/LingYi/20241025181553.png)

In [ ]:
response = client.chat.completions.create(
  model="o1-mini",
  messages=[
      {"role": "user", "content": "请问什么是大模型？"}
  ]
)

In [ ]:
print(response.choices[0].message.content)

怎么选择模型呢？首先要选择最新的旗舰款模型

>- 像 o1 系列模型 这样的推理模型虽然返回结果较慢且使用更多token来"思考"，但它们能进行高级推理、编码和多步骤规划，是目前最智能的模型。这些模型专注于复杂的推理任务和深度思考。
1. **o1-preview**
   - 完整版o1模型预览版（还不是最终的o1，但是目前最智能最贵）
   - 专攻需要广泛常识的复杂问题

2. **o1-mini**
   - 更快速、性价比更高
   - 特别适合编码、数学和科学任务

>- 像 gpt-4o 这样的大型模型虽然每个token成本较高且推理比小模型慢，但能提供非常高水平的智能和强大的性能。

>- 像 gpt-4o-mini 这样的小型模型虽然智能水平不及大型模型，但它们响应速度快，且每个token的成本更低。



messages参数

`messages` 是一个包含多个字典的列表。每个字典代表一条消息，包含两个关键部分：

1. **角色（`role`）**：表示消息的发送者，如用户（`user`）或助手（`assistant`）。
2. **内容（`content`）**：具体的消息内容，即发送的文本。

也就是说，无论是用户的消息，助手回答的消息，最后的结构就是这个字典的消息结构

messages参数中的role设定不仅有“user”这一种，还有“assistant”和“system”，同时还有后面学函数调用的“tool”：

这些角色来源于他们是Message类下的哪个子类：

![](https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/LingYi/20241017111022.png)

**结构解析**：
- `role`: 指定消息的发送者角色（ **但是现在o1只开放了user和assistant，因为系统提示和工具调用的功能还不可用**）
    - “**user**”：表示消息来源于用户，用户向模型提出问题或者发出指令，通常是对话中的输入部分；
    - “**assistant**”：表示消息来自于模型聊天助手，即模型的回答或反馈，通常是对话中的输出部分，包括对用户问题的直接回答、建议或者其他类型的响应；
    - “**system**”：一般用于模型的身份设定。 
    - “**tool**”：函数调用时在本地执行了函数调用后告诉大模型这条消息的内容是函数调用的结果的标志，后面讲函数调用的章节时会具体学到。
- `content`: 单次消息内容列表
  - `type`: 内容类型（文字消息为 "text"，图片消息为"image_url"**目前o1还未开放图像的多模态能力，只有文本**），当消息内容只包含文本时，可以使用简化的格式，`content`的值直接为字符串。但是，如果消息内容包含图像或其他非文本内容，则必须使用完整的格式，包括 "type" 字段。以下两种写法是等效的：
    ```json 
    {"role": "user", "content": "Hello, Claude"}
    {"role": "user", "content": [{"type": "text", "text": "Hello, Claude"}]}
    ```


In [22]:
response_o1 = client.chat.completions.create(
    model="o1-preview-2024-09-12",
    messages=[
      {"role": "user", "content": "量子力学该怎么透彻的学"}
    ],
)

o1响应的结果和gpt4o系列比只有token多了推理token

大模型推理生成的结果直接由create函数返回了，被我们存进了response中

In [23]:
response_o1

ChatCompletion(id='chatcmpl-ANHSQ6HVdtuYafMesD2OQVXQftrVW', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='要透彻地学习量子力学，需要循序渐进、系统深入地进行。以下是一些建议，帮助您更好地掌握量子力学：\n\n1. **强化数学基础**：\n\n   - **线性代数**：熟悉向量空间、矩阵、特征值和特征向量等概念。\n   - **微积分**：熟练掌握微分和积分的技巧，包括多元微积分。\n   - **微分方程**：理解常微分方程和偏微分方程的解法。\n   - **复变函数**：了解复数的性质和复变函数的解析性。\n\n2. **巩固经典物理知识**：\n\n   - **经典力学**：深入理解牛顿力学、拉格朗日力学和哈密顿力学，为量子力学的学习奠定基础。\n   - **电动力学**：掌握麦克斯韦方程组和电磁场理论。\n\n3. **选择合适的教材**：\n\n   - **国内教材**：\n     - 曾谨言的《量子力学教程》\n     - 曾谨言的《量子力学导论》\n   - **国外经典**：\n     - J.J.萨克赖的《现代量子力学》（Modern Quantum Mechanics）\n     - D.J.格里菲斯的《量子力学导论》（Introduction to Quantum Mechanics）\n     - L.D.兰道和E.M.利夫希茨的《量子力学（非相对论理论）》\n     - P.A.M.狄拉克的《量子力学原理》\n\n4. **系统学习理论知识**：\n\n   - **逐章学习**：按照教材章节顺序，系统地学习，从基本概念到复杂应用逐步深入。\n   - **理解物理意义**：不仅要会推导公式，更要理解其背后的物理意义和适用条件。\n   - **重视概念**：量子力学中许多概念与经典物理直观不同，如波函数、叠加原理和测量问题，需要深刻理解。\n\n5. **积极完成习题**：\n\n   - **巩固理论**：通过练习来检验和巩固所学知识。\n   - **提高问题解决能力**：培养分析和解决复杂问题的能

In [24]:
# 查看返回对象类型
type(response_o1)

openai.types.chat.chat_completion.ChatCompletion

模型返回对象是一个类。若想提取response中具体返回的内容，可以直接通过属性调用的方式：

In [25]:
response_o1.id 
# 代表这段对话的唯一标识符。

'chatcmpl-ANHSQ6HVdtuYafMesD2OQVXQftrVW'

API调用中计费的单位是token：

![](https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/LingYi/20241025182906.png)

![](https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/LingYi/20241025182915.png)
- **什么是token**
 - 在大语言模型中,token是文本的基本单位。模型不是直接处理原始文本,而是将文本切分成一系列token进行处理，最后将token映射成数字。token可以是单词、子词或者单个字符,具体取决于模型使用的分词方法。
 - OpenAI 官方提供了一个工具 **Tokenizer**，可以测试任意输入的文本，并查看它们是如何被转换成 Tokens 的。Tokenizer 工具地址：https://platform.openai.com/tokenizer
  - 
    - <center><img src="https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/LingYi/20241010191810.png" alt="Tokenizer 工具" /></center>

可以在返回结果的usage中查看本次对话所占用的token数量。具体含义如下：
- **completion_tokens**：表示生成的回答的token数量；
- **prompt_tokens**：表示输入给模型的prompt占用的token数量（这里指的是“请举几个罗马帝国的趣事”这句话的token数）
- **total_tokens**：表示本次对话总共占用的token数量，是输入和输出token之和。

In [26]:
response_o1.usage

CompletionUsage(completion_tokens=1628, prompt_tokens=19, total_tokens=1647, completion_tokens_details=CompletionTokensDetails(audio_tokens=None, reasoning_tokens=448), prompt_tokens_details=PromptTokensDetails(audio_tokens=None, cached_tokens=0))

模型回应的正文

In [27]:
print(response_o1.choices[0].message.content)

要透彻地学习量子力学，需要循序渐进、系统深入地进行。以下是一些建议，帮助您更好地掌握量子力学：

1. **强化数学基础**：

   - **线性代数**：熟悉向量空间、矩阵、特征值和特征向量等概念。
   - **微积分**：熟练掌握微分和积分的技巧，包括多元微积分。
   - **微分方程**：理解常微分方程和偏微分方程的解法。
   - **复变函数**：了解复数的性质和复变函数的解析性。

2. **巩固经典物理知识**：

   - **经典力学**：深入理解牛顿力学、拉格朗日力学和哈密顿力学，为量子力学的学习奠定基础。
   - **电动力学**：掌握麦克斯韦方程组和电磁场理论。

3. **选择合适的教材**：

   - **国内教材**：
     - 曾谨言的《量子力学教程》
     - 曾谨言的《量子力学导论》
   - **国外经典**：
     - J.J.萨克赖的《现代量子力学》（Modern Quantum Mechanics）
     - D.J.格里菲斯的《量子力学导论》（Introduction to Quantum Mechanics）
     - L.D.兰道和E.M.利夫希茨的《量子力学（非相对论理论）》
     - P.A.M.狄拉克的《量子力学原理》

4. **系统学习理论知识**：

   - **逐章学习**：按照教材章节顺序，系统地学习，从基本概念到复杂应用逐步深入。
   - **理解物理意义**：不仅要会推导公式，更要理解其背后的物理意义和适用条件。
   - **重视概念**：量子力学中许多概念与经典物理直观不同，如波函数、叠加原理和测量问题，需要深刻理解。

5. **积极完成习题**：

   - **巩固理论**：通过练习来检验和巩固所学知识。
   - **提高问题解决能力**：培养分析和解决复杂问题的能力。

6. **参与课程和研讨**：

   - **大学课程**：如果是在校学生，积极参与量子力学课程的学习，充分利用老师的指导。
   - **在线课程**：利用慕课（MOOC）平台，如Coursera、edX、中国大学MOOC等，参加相关在线课程。
   - **研讨班和讲座**：参加物理学术会议、研讨班、专题讲座等，拓宽视野。

7. **与他人交流学习**：

   - **

如何使用 OpenAI O1 模型控制成本

在之前的模型中，我们使用`max_tokens`控制最终可见输出的token数量
为了更准确地反映o1模型的工作方式，及它控制的是"完成过程"中的所有token改为了新的`max_completion_tokens`，它包括：
  * 不可见的推理过程token
  * 最终可见的输出token
    
新的`max_completion_tokens`避免用户误解这个限制只是针对最终输出


1. Token使用的特殊性：
- o1模型使用的token包括可见输出和不可见的推理过程
- 如果达到token限制前未完成推理，将不会返回任何输出
- 需要合理设置max_completion_tokens以保证获得输出

2. 参数变化：
- 新增：max_completion_tokens（取代max_tokens）控制所有完成token的总数（包括推理和输出）
- 建议：至少预留25,000个token用于完成

3. 模型具体限制：
- o1-preview：
  * 上下文窗口：128,000 tokens
  * 输出上限：32,768 tokens

- o1-mini：
  * 上下文窗口：128,000 tokens
  * 输出上限：65,536 tokens

4. 成本控制建议：
- 需要根据具体用例测试合适的max_completion_tokens值
- 注意监控上下文窗口使用情况，避免输出被截断
- 合理设置以避免支付无效的推理token费用

让我们在前面的示例中尝试max_completion_tokens 。我们将其设置为100 ，看看会得到什么。要使用这个新参数，

In [ ]:
response_o1 = client.chat.completions.create(
    model="o1-mini",
    messages=[
      {"role": "user", "content": "请帮我写3个问句"}
    ],
    max_completion_tokens=100
)


因为长度停止，所以是空白回复没有生成

In [18]:
response_o1

ChatCompletion(id='chatcmpl-ANFxcXYj2bLAgbOiBDPTc8GPWKqnl', choices=[Choice(finish_reason='length', index=0, logprobs=None, message=ChatCompletionMessage(content='', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1730106260, model='o1-mini-2024-09-12', object='chat.completion', service_tier=None, system_fingerprint='fp_93b09d5093', usage=CompletionUsage(completion_tokens=100, prompt_tokens=16, total_tokens=116, completion_tokens_details=CompletionTokensDetails(audio_tokens=None, reasoning_tokens=100), prompt_tokens_details=PromptTokensDetails(audio_tokens=None, cached_tokens=0)))

API调用区别与总结

In [ ]:
import openai
client = openai.OpenAI()

In [20]:
response_o1 = client.chat.completions.create(
    model="o1-mini",
    messages=[
      {"role": "user", "content": "请帮我写3个问句"}
    ],
)


response_4o = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
      {"role": "user", "content": "请帮我写3个问句"}
    ],
)


![](https://typora-photo1220.oss-cn-beijing.aliyuncs.com/DataAnalysis/LingYi/20241022185124.png)

比较两个模型的回复，都是同一个类的对象，没有差别，只有token处会有区别。

In [21]:
# response_o1 = client.chat.completions.create(model="o1-mini", messages=[{"role": "user", "content": "请帮我写3个问句"}])
# response_4o = client.chat.completions.create(model="gpt-4o-mini", messages=[{"role": "user", "content": "请帮我写3个问句"}])

def compare_token_usage(response1, response2, label1='o1-mini', label2='gpt-4o-mini'):
    """
    比较两个ChatCompletion响应对象的令牌使用情况，并打印结果。
    """
    # 提取令牌使用数据
    tokens = ['completion_tokens', 'prompt_tokens', 'total_tokens']
    
    print(f"{'属性':<20} | {label1:<15} | {label2:<15} | {'差异':<10}")
    print("-" * 65)
    
    for token in tokens:
        value1 = getattr(response1.usage, token, 'N/A')
        value2 = getattr(response2.usage, token, 'N/A')
        if isinstance(value1, int) and isinstance(value2, int):
            difference = value2 - value1
        else:
            difference = 'N/A'
        print(f"{token:<20} | {value1:<15} | {value2:<15} | {difference:<10}")

compare_token_usage(response_o1, response_4o)


属性                   | o1-mini         | gpt-4o-mini     | 差异        
-----------------------------------------------------------------
completion_tokens    | 244             | 38              | -206      
prompt_tokens        | 16              | 15              | -1        
total_tokens         | 260             | 53              | -207      
